# Deep Agents: Building Complex Agents for Long-Horizon Tasks

In this notebook, we'll explore **Deep Agents** - a new approach to building AI agents that can handle complex, multi-step tasks over extended periods. We'll implement all four key elements of Deep Agents while building on our Personal Wellness Assistant use case.

**Learning Objectives:**
- Understand the four key elements of Deep Agents: Planning, Context Management, Subagent Spawning, and Long-term Memory
- Implement each element progressively using the `deepagents` package
- Learn to use Skills for progressive capability disclosure
- Use the `deepagents-cli` for interactive agent sessions

## Table of Contents:

- **Breakout Room #1:** Deep Agent Foundations
  - Task 1: Dependencies & Setup
  - Task 2: Understanding Deep Agents
  - Task 3: Planning with Todo Lists
  - Task 4: Context Management with File Systems
  - Task 5: Basic Deep Agent
  - Question #1 & Question #2
  - Activity #1: Build a Research Agent

- **Breakout Room #2:** Advanced Features & Integration
  - Task 6: Subagent Spawning
  - Task 7: Long-term Memory Integration
  - Task 8: Skills - On-Demand Capabilities
  - Task 9: Using deepagents-cli
  - Task 10: Building a Complete Deep Agent System
  - Question #3 & Question #4
  - Activity #2: Build a Wellness Coach Agent

---
# 🤝 Breakout Room #1
## Deep Agent Foundations

## Task 1: Dependencies & Setup

Before we begin, make sure you have:

1. **API Keys** for:
   - Anthropic (default for Deep Agents) or OpenAI
   - LangSmith (optional, for tracing)
   - Tavily (optional, for web search)

2. **Dependencies installed** via `uv sync`

3. **For the CLI** (Task 9): `uv pip install deepagents-cli`

### Environment Setup

You can either:
- Create a `.env` file with your API keys (recommended):
  ```
  ANTHROPIC_API_KEY=your_key_here
  OPENAI_API_KEY=your_key_here
  LANGCHAIN_API_KEY=your_key_here
  ```
- Or enter them interactively when prompted

In [10]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict, Literal

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

In [11]:
# Set Anthropic API Key (default for Deep Agents)
anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

Anthropic API key set


In [12]:
# Optional: LangSmith for tracing
langsmith_key = get_api_key("LANGCHAIN_API_KEY", "LangSmith API Key (press Enter to skip): ")

if langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Deep Agents - {uuid4().hex[0:8]}"
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")

LangSmith tracing enabled. Project: AIE9 - Deep Agents - 94f9c81e


In [13]:
!pip install deepagents

In [14]:
# Verify deepagents installation
from deepagents import create_deep_agent
print("deepagents package imported successfully!")

# Test with a simple agent
test_agent = create_deep_agent()
result = test_agent.invoke({
    "messages": [{"role": "user", "content": "Say 'Deep Agents ready!' in exactly those words."}]
})
print(result["messages"][-1].content)

deepagents package imported successfully!


c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Pyt

Deep Agents ready!


## Task 2: Understanding Deep Agents

**Deep Agents** represent a shift from simple tool-calling loops to sophisticated agents that can handle complex, long-horizon tasks. They address four key challenges:

### The Four Key Elements

| Element | Challenge Addressed | Implementation |
|---------|---------------------|----------------|
| **Planning** | "What should I do?" | Todo lists that persist task state |
| **Context Management** | "What do I know?" | File systems for storing/retrieving info |
| **Subagent Spawning** | "Who can help?" | Task tool for delegating to specialists |
| **Long-term Memory** | "What did I learn?" | LangGraph Store for cross-session memory |

### Deep Agents vs Traditional Agents

```
Traditional Agent Loop:
┌─────────────────────────────────────┐
│  User Query                         │
│       ↓                             │
│  Think → Act → Observe → Repeat     │
│       ↓                             │
│  Response                           │
└─────────────────────────────────────┘
Problems: Context bloat, no delegation,
          loses track of complex tasks

Deep Agent Architecture:
┌─────────────────────────────────────────────────────────┐
│                    Deep Agent                           │
├─────────────────────────────────────────────────────────┤
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐   │
│  │   PLANNING   │  │   CONTEXT    │  │   MEMORY     │   │
│  │              │  │  MANAGEMENT  │  │              │   │
│  │ write_todos  │  │              │  │   Store      │   │
│  │ update_todo  │  │  read_file   │  │  namespace   │   │
│  │ list_todos   │  │  write_file  │  │  get/put     │   │
│  │              │  │  edit_file   │  │              │   │
│  └──────────────┘  │  ls          │  └──────────────┘   │
│                    └──────────────┘                     │
│  ┌──────────────────────────────────────────────────┐   │
│  │              SUBAGENT SPAWNING                   │   │
│  │                                                  │   │
│  │  task(prompt, tools, model, system_prompt)       │   │
│  │       ↓              ↓              ↓            │   │
│  │  ┌────────┐    ┌────────┐    ┌────────┐          │   │
│  │  │Research│    │Writing │    │Analysis│          │   │
│  │  │Subagent│    │Subagent│    │Subagent│          │   │
│  │  └────────┘    └────────┘    └────────┘          │   │
│  └──────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────┘
```

### When to Use Deep Agents

| Use Case | Traditional Agent | Deep Agent |
|----------|-------------------|------------|
| Simple Q&A | ✅ | Overkill |
| Single-step tool use | ✅ | Overkill |
| Multi-step research | ⚠️ May lose track | ✅ |
| Complex projects | ❌ Context overflow | ✅ |
| Parallel task execution | ❌ | ✅ |
| Long-running sessions | ❌ | ✅ |

### Key Insight: "Planning is Context Engineering"

Deep Agents treat planning not as a separate phase, but as **context engineering**:
- Todo lists aren't just task trackers—they're **persistent context** about what to do
- File systems aren't just storage—they're **extended memory** beyond the context window
- Subagents aren't just helpers—they're **context isolation** to prevent bloat

## Task 3: Planning with Todo Lists

The first key element of Deep Agents is **Planning**. Instead of trying to hold all task state in the conversation, Deep Agents use structured todo lists.

### Why Todo Lists?

1. **Persistence**: Tasks survive across conversation turns
2. **Visibility**: Both agent and user can see progress
3. **Structure**: Clear tracking of what's done vs pending
4. **Recovery**: Agent can resume from where it left off

### Todo List Tools

| Tool | Purpose |
|------|----------|
| `write_todos` | Create a structured task list |
| `update_todo` | Mark tasks as complete/in-progress |
| `list_todos` | View current task state |

In [15]:
from langchain_core.tools import tool
from typing import List, Optional
import json

# Simple in-memory todo storage for demonstration
# In production, Deep Agents use persistent storage
TODO_STORE = {}

@tool
def write_todos(todos: List[dict]) -> str:
    """Create a list of todos for tracking task progress.
    
    Args:
        todos: List of todo items, each with 'title' and optional 'description'
    
    Returns:
        Confirmation message with todo IDs
    """
    created = []
    for i, todo in enumerate(todos):
        todo_id = f"todo_{len(TODO_STORE) + i + 1}"
        TODO_STORE[todo_id] = {
            "id": todo_id,
            "title": todo.get("title", "Untitled"),
            "description": todo.get("description", ""),
            "status": "pending"
        }
        created.append(todo_id)
    return f"Created {len(created)} todos: {', '.join(created)}"

@tool
def update_todo(todo_id: str, status: Literal["pending", "in_progress", "completed"]) -> str:
    """Update the status of a todo item.
    
    Args:
        todo_id: The ID of the todo to update
        status: New status (pending, in_progress, completed)
    
    Returns:
        Confirmation message
    """
    if todo_id not in TODO_STORE:
        return f"Todo {todo_id} not found"
    TODO_STORE[todo_id]["status"] = status
    return f"Updated {todo_id} to {status}"

@tool
def list_todos() -> str:
    """List all todos with their current status.
    
    Returns:
        Formatted list of all todos
    """
    if not TODO_STORE:
        return "No todos found"
    
    result = []
    for todo_id, todo in TODO_STORE.items():
        status_emoji = {"pending": "⬜", "in_progress": "🔄", "completed": "✅"}
        emoji = status_emoji.get(todo["status"], "❓")
        result.append(f"{emoji} [{todo_id}] {todo['title']} ({todo['status']})")
    return "\n".join(result)

print("Todo tools defined!")

Todo tools defined!


In [16]:
# Test the todo tools
TODO_STORE.clear()  # Reset for demo

# Create some wellness todos
result = write_todos.invoke({
    "todos": [
        {"title": "Assess current sleep patterns", "description": "Review user's sleep schedule and quality"},
        {"title": "Research sleep improvement strategies", "description": "Find evidence-based techniques"},
        {"title": "Create personalized sleep plan", "description": "Combine findings into actionable steps"},
    ]
})
print(result)
print("\nCurrent todos:")
print(list_todos.invoke({}))

Created 3 todos: todo_1, todo_3, todo_5

Current todos:
⬜ [todo_1] Assess current sleep patterns (pending)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


In [17]:
# Simulate progress
update_todo.invoke({"todo_id": "todo_1", "status": "completed"})
update_todo.invoke({"todo_id": "todo_2", "status": "in_progress"})

print("After updates:")
print(list_todos.invoke({}))

After updates:
✅ [todo_1] Assess current sleep patterns (completed)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


## Task 4: Context Management with File Systems

The second key element is **Context Management**. Deep Agents use file systems to:

1. **Offload large content** - Store research, documents, and results to disk
2. **Persist across sessions** - Files survive beyond conversation context
3. **Share between subagents** - Subagents can read/write shared files
4. **Prevent context overflow** - Large tool results automatically saved to disk

### Automatic Context Management

Deep Agents automatically handle context limits:
- **Large result offloading**: Tool results >20k tokens → saved to disk
- **Proactive offloading**: At 85% context capacity → agent saves state to disk
- **Summarization**: Long conversations get summarized while preserving intent

### File System Tools

| Tool | Purpose |
|------|----------|
| `ls` | List directory contents |
| `read_file` | Read file contents |
| `write_file` | Create/overwrite files |
| `edit_file` | Make targeted edits |

In [18]:
import os
from pathlib import Path

# Create a workspace directory for our agent
WORKSPACE = Path("workspace")
WORKSPACE.mkdir(exist_ok=True)

@tool
def ls(path: str = ".") -> str:
    """List contents of a directory.
    
    Args:
        path: Directory path to list (default: current directory)
    
    Returns:
        List of files and directories
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"Directory not found: {path}"
    
    items = []
    for item in sorted(target.iterdir()):
        prefix = "[DIR]" if item.is_dir() else "[FILE]"
        size = f" ({item.stat().st_size} bytes)" if item.is_file() else ""
        items.append(f"{prefix} {item.name}{size}")
    
    return "\n".join(items) if items else "(empty directory)"

@tool
def read_file(path: str) -> str:
    """Read contents of a file.
    
    Args:
        path: Path to the file to read
    
    Returns:
        File contents
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    return target.read_text()

@tool
def write_file(path: str, content: str) -> str:
    """Write content to a file (creates or overwrites).
    
    Args:
        path: Path to the file to write
        content: Content to write to the file
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    target.parent.mkdir(parents=True, exist_ok=True)
    target.write_text(content)
    return f"Wrote {len(content)} characters to {path}"

@tool
def edit_file(path: str, old_text: str, new_text: str) -> str:
    """Edit a file by replacing text.
    
    Args:
        path: Path to the file to edit
        old_text: Text to find and replace
        new_text: Replacement text
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    
    content = target.read_text()
    if old_text not in content:
        return f"Text not found in {path}"
    
    new_content = content.replace(old_text, new_text, 1)
    target.write_text(new_content)
    return f"Updated {path}"

print("File system tools defined!")
print(f"Workspace: {WORKSPACE.absolute()}")

File system tools defined!
Workspace: c:\MyWorkspace\Assignments\AIE9\07_Deep_Agents\workspace


In [19]:
# Test the file system tools
print("Current workspace contents:")
print(ls.invoke({"path": "."}))

Current workspace contents:
[FILE] morning_energy_quick_guide.md (6294 bytes)
[FILE] morning_exercise_quick_guide.md (5309 bytes)
[FILE] morning_exercise_research.md (15656 bytes)
[FILE] morning_mindset_practices_guide.md (19833 bytes)
[FILE] morning_nutrition_research.md (20218 bytes)
[FILE] morning_routine_science_report.md (12185 bytes)
[FILE] personalized_sleep_improvement_plan.md (5749 bytes)
[DIR] research


In [20]:
# Create a research notes file
notes = """# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations
"""

result = write_file.invoke({"path": "research/sleep_notes.md", "content": notes})
print(result)

# Verify it was created
print("\nResearch directory:")
print(ls.invoke({"path": "research"}))

Wrote 242 characters to research/sleep_notes.md

Research directory:
[FILE] comprehensive_sleep_research.md (12179 bytes)
[FILE] sleep_improvement_plan_summary.md (9484 bytes)
[FILE] sleep_notes.md (252 bytes)


In [21]:
# Read and edit the file
print("File contents:")
print(read_file.invoke({"path": "research/sleep_notes.md"}))

File contents:
# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations



## Task 5: Basic Deep Agent

Now let's create a basic Deep Agent using the `deepagents` package. This combines:
- Planning (todo lists)
- Context management (file system)
- A capable LLM backbone

### Configuring the FilesystemBackend

Deep Agents come with **built-in file tools** (`ls`, `read_file`, `write_file`, `edit_file`). To control where files are stored, we configure a `FilesystemBackend`:

```python
from deepagents.backends import FilesystemBackend

backend = FilesystemBackend(
    root_dir="/path/to/workspace",
    virtual_mode=True  # REQUIRED to actually sandbox files!
)
```

**Critical: `virtual_mode=True`**
- Without `virtual_mode=True`, agents can still write anywhere on the filesystem!
- The `root_dir` alone does NOT restrict file access
- `virtual_mode=True` blocks paths with `..`, `~`, and absolute paths outside root

In [22]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Configure the filesystem backend to use our workspace directory
# IMPORTANT: virtual_mode=True is required to actually restrict paths to root_dir
# Without it, agents can still write anywhere on the filesystem!
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
]

# Create a basic Deep Agent
wellness_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where files are stored
    system_prompt="""You are a Personal Wellness Assistant that helps users improve their health.

When given a complex task:
1. First, create a todo list to track your progress
2. Work through each task, updating status as you go
3. Save important findings to files for reference
4. Provide a clear summary when complete

Be thorough but concise. Always explain your reasoning."""
)

print(f"Basic Deep Agent created!")
print(f"File operations sandboxed to: {workspace_path}")

c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Pyt

Basic Deep Agent created!
File operations sandboxed to: c:\MyWorkspace\Assignments\AIE9\07_Deep_Agents\workspace


In [19]:
# Reset todo store for fresh demo
TODO_STORE.clear()

# Test with a multi-step wellness task
result = wellness_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please create a personalized sleep improvement plan for me and save it to a file."""
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## ✅ Your Personalized Sleep Improvement Plan is Complete!

I've created a comprehensive 12-week sleep improvement plan specifically tailored to your challenges and saved it to `/personalized_sleep_improvement_plan.md`. 

### 🎯 **Your Plan Addresses:**
- **Inconsistent bedtime:** Gradual 15-minute adjustments every 2 days toward a target time
- **Phone use in bed:** Complete phone-free bedroom strategy with practical alternatives
- **Morning fatigue:** Sleep environment optimization and morning light therapy

### 🚀 **Start This Week With These 3 Simple Steps:**

1. **Choose your target bedtime** (I recommend 10:30pm as a middle ground)
2. **Set up phone charging station** outside your bedroom 
3. **Get 10-15 minutes of bright light** within 1 hour of waking up

### 📊 **What Makes This Plan Special:**
- **Gradual approach:** No dramatic overnight changes
- **Evidence-based:** All strategies backed by sleep science
- **Personalized tracking:** Daily scorecard system
- **R

In [21]:
# Check what the agent created
print("Todo list after task:")
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")

Todo list after task:
✅ [todo_1] Analyze current sleep issues (completed)
✅ [todo_3] Research evidence-based sleep improvement strategies (completed)
✅ [todo_5] Create personalized sleep improvement plan (completed)
✅ [todo_7] Save plan to file (completed)
✅ [todo_9] Provide implementation guidance (completed)
✅ [todo_6] Research sleep science fundamentals and circadian rhythm regulation (completed)
✅ [todo_8] Research evidence-based strategies for establishing consistent sleep schedule (completed)
✅ [todo_10] Research blue light and screen impact on sleep (completed)
✅ [todo_12] Research sleep quality improvement techniques (completed)
✅ [todo_14] Research sleep hygiene and environmental factors (completed)
✅ [todo_16] Research implementation timelines and progression strategies (completed)
✅ [todo_18] Research measurable goals and tracking methods (completed)
✅ [todo_20] Compile and structure comprehensive research summary (completed)


Workspace contents:
  [FILE] personalized_sleep

---
## ❓ Question #1:

What are the **trade-offs** of using todo lists for planning? Consider:
- When might explicit planning overhead slow things down?
- How granular should todo items be?
- What happens if the agent creates todos but never completes them?

##### Answer:
*For simple, single-step questions (“What’s a good post-workout meal?”), writing todos adds extra tool calls + tokens for no real gain.

If the agent re-plans too often (rewrite todos every turn), it creates thrash: planning becomes the task.


How granular should todos be

1. Best: 1–5 minute, “checkable” steps that produce an observable output.

2. Good: “Extract constraints (diet, schedule)”, “Generate 3 options”, “Pick one + summarize”

3. Too coarse: “Improve lifestyle”

4. Too fine: “Think about breakfast… think about lunch…”


Rule: if a todo can’t be marked done with a clear artifact (text, file, decision), it’s too vague.


What if the agent creates todos but never completes them

1. You get illusion of progress: lots of planning, no delivery.

2. Users lose trust because the agent “organizes” but doesn’t execute.

Fixes (good to mention):

1. Enforce a “do one todo before adding more” rule

2. Add periodic review: “What’s done / next / blocked?”*

## ❓ Question #2:

How would you design a **context management strategy** for a wellness agent that:
- Needs to reference a large health document (16KB)
- Tracks user metrics over time
- Must remember user conditions (allergies, medications) for safety

What goes in files vs. in the prompt? What should never be offloaded?

##### Answer:
*What goes in files (offload)

1) The large health document (16KB)

Store as /docs/health_doc.md

Also store a structured summary /summaries/health_doc_summary.md

Use retrieval (or targeted reads) to pull only the relevant section per question.


2) User metrics over time

Store as append-only logs or tables:

/metrics/weekly.csv (weight, steps, sleep, water, mood, etc.)

/metrics/notes.md (freeform notes)


Keep a small derived “trend snapshot” file:

/metrics/latest_summary.md (last 7–14 days trends)



3) Long-lived profile data

/profile/user_profile.json (age range, goals, preferences)

/profile/conditions.json (conditions, meds, allergies — canonical source)


What goes in the prompt (keep small + always present)

A short Safety Card (always included every run):

allergies, medications, critical conditions

“never recommend contraindicated items” constraints


The current user goal and current plan (1–2 paragraphs)

The recent window of metrics (e.g., last 7 days summary), not the whole history


What should never be offloaded (must always be in active context)

Safety-critical constraints at decision time (allergies/meds/conditions):
You can store them in files, but the agent must always load and include them in the working prompt before giving advice.

System / policy rules (guardrails like “don’t give medical diagnosis”, “recommend seeing a clinician when needed”)

Anything required to avoid harmful suggestions (e.g., “no NSAIDs due to anticoagulants” type constraints)


A simple, effective pattern to describe

Keep the prompt lean: Safety Card + Goal + Last-week metrics + Question

Everything else (big doc + long metric history) lives in files and is retrieved/loaded only when needed

Maintain two summaries:

“Doc summary” (stable)

“User snapshot” (updated every session)*

---
## 🏗️ Activity #1: Build a Research Agent

Build a Deep Agent that can research a wellness topic and produce a structured report.

### Requirements:
1. Create todos for the research process
2. Read from the HealthWellnessGuide.txt in the data folder
3. Save findings to a structured markdown file
4. Update todo status as tasks complete

### Test prompt:
"Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."

In [23]:
# --- Activity #1: Research Agent (Deep Agents) ---

from pathlib import Path
from typing import List
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend

# ----------------------------
# 0) Todo tools (from notebook)
# ----------------------------
# Assumes you already ran the notebook cells that define:
# TODO_STORE, write_todos, update_todo, list_todos
# If not, run those earlier cells first.

# ----------------------------
# 1) Filesystem backend sandbox
# ----------------------------
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True, # IMPORTANT: sandbox file ops
)

# ----------------------------
# 2) Tool: Read wellness guide (excerpt-based)
# ----------------------------
@tool
def read_health_wellness_guide(query: str = "", max_chars: int = 12000) -> str:
    """
    Read HealthWellnessGuide.txt from the data folder.
    If query is provided, return only relevant excerpts (simple keyword match) to reduce context bloat.
    """
    file_path = Path("data/HealthWellnessGuide.txt")
    if not file_path.exists():
        return "ERROR: data/HealthWellnessGuide.txt not found."

    text = file_path.read_text(encoding="utf-8", errors="ignore")

    q = (query or "").strip().lower()
    if not q:
        return text[:max_chars]

    # Simple excerpt extraction: return paragraphs containing any query tokens
    tokens = [t for t in q.replace(",", " ").split() if len(t) >= 4]
    if not tokens:
        return text[:max_chars]

    paras = [p.strip() for p in text.split("\n\n") if p.strip()]
    hits = []
    for p in paras:
        pl = p.lower()
        if any(t in pl for t in tokens):
            hits.append(p)
        if sum(len(x) for x in hits) >= max_chars:
            break

    if not hits:
        # fallback: return top of doc if nothing matched
        return text[:max_chars]

    out = "\n\n---\n\n".join(hits)
    return out[:max_chars]

# ----------------------------
# 3) Build the research agent
# ----------------------------
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
    read_health_wellness_guide,
    # Deep Agents already provides file tools (write_file/read_file/ls/edit_file)
    # through the backend, so you don't need to define them here.
]

research_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,
    system_prompt="""
You are a Wellness Research Agent.

Goal: Research the user's wellness topic using ONLY the provided HealthWellnessGuide.txt (via read_health_wellness_guide).
Then write a structured markdown report into the workspace folder.

Process (mandatory):
1) Create a todo list for the research workflow.
2) Read relevant excerpts from HealthWellnessGuide.txt (use query terms).
3) Draft a report with: Overview, 5+ evidence-based strategies, How to apply, Safety notes, References (quote/point to guide excerpts).
4) Save the report as a markdown file in workspace (e.g., stress_management_guide.md).
5) Update todo statuses as tasks complete.

Output: After saving, tell the user the filename and a short summary of what’s inside.
Do NOT include hidden reasoning. Just provide the report and evidence-based explanations.
""".strip()
)

# ----------------------------
# 4) Test prompt
# ----------------------------
TODO_STORE.clear()

test_prompt = "Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."

result = research_agent.invoke({
    "messages": [{"role": "user", "content": test_prompt}]
})

print(result["messages"][-1].content)

print("\n--- TODOS ---")
print(list_todos.invoke({}))

print("\n--- WORKSPACE FILES ---")
for f in sorted(workspace_path.iterdir()):
    print(f"- {f.name}")

c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Pyt

## Research Complete!

I've successfully created a comprehensive stress management guide saved as **`stress_management_guide.md`** in your workspace folder.

**What's inside the guide:**
- **7 evidence-based strategies** including deep breathing, progressive muscle relaxation, mindfulness meditation, grounding techniques, exercise, sleep hygiene, and social connection
- **Practical application sections** with emergency stress relief (5-minute techniques), daily routines, and long-term management approaches
- **Scientific rationale** for each technique explaining why it works
- **Safety considerations** and when to seek professional help
- **Daily wellness checklist** for ongoing stress management
- **Step-by-step instructions** for implementing each technique

The guide is based entirely on evidence from the HealthWellnessGuide.txt and provides actionable strategies you can start using immediately, from 5-minute emergency techniques to comprehensive lifestyle approaches for long-term s

---
# 🤝 Breakout Room #2
## Advanced Features & Integration

## Task 6: Subagent Spawning

The third key element is **Subagent Spawning**. This allows a Deep Agent to delegate tasks to specialized subagents.

### Why Subagents?

1. **Context Isolation**: Each subagent has its own context window, preventing bloat
2. **Specialization**: Different subagents can have different tools/prompts
3. **Parallelism**: Multiple subagents can work simultaneously
4. **Cost Optimization**: Use cheaper models for simpler subtasks

### How Subagents Work

```
Main Agent
    ├── task("Research sleep science", model="gpt-4o-mini")
    │       └── Returns: Summary of findings
    │
    ├── task("Analyze user's sleep data", tools=[analyze_tool])
    │       └── Returns: Analysis results
    │
    └── task("Write recommendations", system_prompt="Be concise")
            └── Returns: Final recommendations
```

Key benefit: The main agent only receives **summaries**, not all the intermediate context!

In [27]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Define specialized subagent configurations
# Note: Subagents inherit the backend from the parent agent
research_subagent = {
    "name": "research-agent",
    "description": "Use this agent to research wellness topics in depth. It can read documents and synthesize information.",
    "system_prompt": """You are a wellness research specialist. Your job is to:
1. Find relevant information in provided documents
2. Synthesize findings into clear summaries
3. Cite sources when possible

Be thorough but concise. Focus on evidence-based information.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",  # Cheaper model for research
}

writing_subagent = {
    "name": "writing-agent",
    "description": "Use this agent to create well-structured documents, plans, and guides.",
    "system_prompt": """You are a wellness content writer. Your job is to:
1. Take research findings and turn them into clear, actionable content
2. Structure information for easy understanding
3. Use formatting (headers, bullets, etc.) effectively

Write in a supportive, encouraging tone.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "anthropic:claude-sonnet-4-20250514",
}

print("Subagent configurations defined!")

Subagent configurations defined!


In [29]:
anthropic_model = init_chat_model("anthropic:claude-sonnet-4-20250514")

research_subagent = {
    "name": "research-agent",
    "description": "Finds evidence-based wellness information from HealthWellnessGuide and produces bullet-point findings with citations.",
    "model": anthropic_model,
    "tools": [read_health_wellness_guide, write_file, read_file, ls],
    "system_prompt": (
        "You are a wellness researcher. Use read_health_wellness_guide to extract evidence-based "
        "stress management techniques. Return: (1) key findings bullets, (2) citations/quotes from the guide, "
        "(3) suggested outline for the final report."
    ),
}

writing_subagent = {
    "name": "writing-agent",
    "description": "Turns research notes into a structured markdown wellness report and saves it to a file.",
    "model": anthropic_model,
    "tools": [write_file, read_file, ls],
    "system_prompt": (
        "You are a wellness writer. Take research notes and produce a structured markdown guide "
        "with headings, numbered strategies, and actionable steps. Save the final report as a .md file."
    ),
}

coordinator_agent = create_deep_agent(
    model=anthropic_model,
    tools=[write_todos, update_todo, list_todos],
    backend=filesystem_backend,
    subagents=[research_subagent, writing_subagent],
    system_prompt=(
        "You are a Wellness Project Coordinator.\n"
        "1) Create todos for the workflow.\n"
        "2) Ask research-agent to gather evidence-based strategies from the guide.\n"
        "3) Ask writing-agent to write a structured markdown report and save it.\n"
        "4) Update todos as each step completes.\n"
        "Always finish with the filename of the report."
    ),
)

print("✅ Coordinator agent created with subagents!")


c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Pyt

✅ Coordinator agent created with subagents!


In [31]:
from pathlib import Path
from deepagents.backends import FilesystemBackend

workspace_path = Path.cwd() / "workspace"
workspace_path.mkdir(parents=True, exist_ok=True)

filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True,
)

# 2) Add a safe tool that can ONLY write inside workspace

from langchain_core.tools import tool

@tool
def save_markdown_report(filename: str, content: str) -> str:
    """
    Save a markdown report inside the workspace folder ONLY.
    filename must be a simple name like 'morning_routine.md' (no slashes).
    """
    if "/" in filename or "\\" in filename or ".." in filename:
        return "ERROR: filename must not contain path separators or '..'"

    if not filename.lower().endswith(".md"):
        filename = filename + ".md"

    out_path = Path.cwd() / "workspace" / filename
    out_path.write_text(content, encoding="utf-8")
    return f"Saved: {out_path.name}"

# 3) Update subagents to use the safe tool (and stop giving them raw file tools)

anthropic_model = init_chat_model("anthropic:claude-sonnet-4-20250514")

research_subagent = {
    "name": "research-agent",
    "description": "Researches from HealthWellnessGuide and returns evidence + bullets.",
    "model": anthropic_model,
    "tools": [read_health_wellness_guide],
    "system_prompt": "Use read_health_wellness_guide. Return evidence-backed bullets + outline.",
}

writing_subagent = {
    "name": "writing-agent",
    "description": "Writes a clean markdown guide and saves it to workspace.",
    "model": anthropic_model,
    "tools": [save_markdown_report],
    "system_prompt": (
        "Write a well-formatted markdown guide. Then call save_markdown_report "
        "with filename='morning_routine_guide.md'. Do not try to write anywhere else."
    ),
}

coordinator_agent = create_deep_agent(
    model=anthropic_model,
    tools=[write_todos, update_todo, list_todos],
    backend=filesystem_backend,
    subagents=[research_subagent, writing_subagent],
    system_prompt=(
        "Create todos. Ask research-agent for evidence + outline. "
        "Then ask writing-agent to write markdown and save it. "
        "Update todos as you go. Finish by stating the saved filename."
    ),
)


c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Pyt

In [33]:
# Check the results
print("Final todo status:")
print(list_todos.invoke({}))

print("\nGenerated files in workspace:")
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

Final todo status:
⬜ [todo_1] Research evidence-based morning routine strategies (pending)
⬜ [todo_3] Create comprehensive markdown report (pending)
⬜ [todo_5] Verify report completion and provide filename (pending)

Generated files in workspace:
  [FILE] morning_energy_quick_guide.md (6294 bytes)
  [FILE] morning_exercise_quick_guide.md (5309 bytes)
  [FILE] morning_exercise_research.md (15656 bytes)
  [FILE] morning_mindset_practices_guide.md (19833 bytes)
  [FILE] morning_nutrition_research.md (20218 bytes)
  [FILE] morning_routine_science_report.md (12185 bytes)
  [FILE] personalized_sleep_improvement_plan.md (5749 bytes)
  [DIR] research/
  [DIR] workspace/


## Task 7: Long-term Memory Integration

The fourth key element is **Long-term Memory**. Deep Agents integrate with LangGraph's Store for persistent memory across sessions.

### Memory Types in Deep Agents

| Type | Scope | Use Case |
|------|-------|----------|
| **Thread Memory** | Single conversation | Current session context |
| **User Memory** | Across threads, per user | User preferences, history |
| **Shared Memory** | Across all users | Common knowledge, learned patterns |

### Integration with LangGraph Store

Deep Agents can use the same `InMemoryStore` (or `PostgresStore`) we learned in Session 6:

In [40]:
from langgraph.store.memory import InMemoryStore

# Create a memory store
memory_store = InMemoryStore()

# Store user profile
user_id = "user_krishanu"
profile_namespace = (user_id, "profile")

memory_store.put(profile_namespace, "name", {"value": "krishanu"})
memory_store.put(profile_namespace, "goals", {
    "primary": "improve energy levels",
    "secondary": "better focus during work hours"
})
memory_store.put(profile_namespace, "conditions", {
    "dietary": ["Indian Non-vegetarian"],
    "medical": ["mild anxiety"]
})
memory_store.put(profile_namespace, "preferences", {
    "exercise_time": "late evening",
    "communication_style": "Short and concise"
})

print(f"Stored profile for {user_id}")

# Retrieve and display
for item in memory_store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

Stored profile for user_krishanu
  name: {'value': 'krishanu'}
  goals: {'primary': 'improve energy levels', 'secondary': 'better focus during work hours'}
  conditions: {'dietary': ['Indian Non-vegetarian'], 'medical': ['mild anxiety']}
  preferences: {'exercise_time': 'late evening', 'communication_style': 'Short and concise'}


In [41]:
# Create memory-aware tools
from langgraph.store.base import BaseStore

@tool
def get_user_profile(user_id: str) -> str:
    """Retrieve a user's wellness profile from long-term memory.
    
    Args:
        user_id: The user's unique identifier
    
    Returns:
        User profile as formatted text
    """
    namespace = (user_id, "profile")
    items = list(memory_store.search(namespace))
    
    if not items:
        return f"No profile found for {user_id}"
    
    result = [f"Profile for {user_id}:"]
    for item in items:
        result.append(f"  {item.key}: {item.value}")
    return "\n".join(result)

@tool
def save_user_preference(user_id: str, key: str, value: str) -> str:
    """Save a user preference to long-term memory.
    
    Args:
        user_id: The user's unique identifier
        key: The preference key
        value: The preference value
    
    Returns:
        Confirmation message
    """
    namespace = (user_id, "preferences")
    memory_store.put(namespace, key, {"value": value})
    return f"Saved preference '{key}' for {user_id}"

print("Memory tools defined!")

Memory tools defined!


In [42]:
# Create a memory-enhanced agent
memory_tools = [
    get_user_profile,
    save_user_preference,
    write_todos,
    update_todo,
    list_todos,
]

memory_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=memory_tools,
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a Personal Wellness Assistant with long-term memory.

At the start of each conversation:
1. Check the user's profile to understand their goals and conditions
2. Personalize all advice based on their profile
3. Save any new preferences they mention

Always reference stored information to show you remember the user."""
)

print("Memory-enhanced agent created!")

c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Pyt

Memory-enhanced agent created!


In [43]:
# Test the memory agent
TODO_STORE.clear()

result = memory_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Hi! My user_id is user_alex. What exercise routine would you recommend for me?"
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Welcome! Since this appears to be our first interaction, I don't have any information about your current fitness level, goals, health conditions, or preferences yet. To recommend the best exercise routine for you, I'd love to learn more about:

1. **Current fitness level** - Are you a beginner, intermediate, or advanced exerciser?
2. **Goals** - Are you looking to lose weight, build muscle, improve cardiovascular health, increase flexibility, or something else?
3. **Available time** - How many days per week and minutes per session can you dedicate to exercise?
4. **Preferences** - Do you prefer gym workouts, home exercises, outdoor activities, group classes, etc.?
5. **Any limitations** - Do you have any injuries, health conditions, or physical limitations I should consider?
6. **Equipment access** - What equipment do you have available (gym membership, home equipment, etc.)?

Once you share some of these details, I'll save them to your profile and create a personalized

## Task 8: Skills - On-Demand Capabilities

**Skills** are a powerful feature for progressive capability disclosure. Instead of loading all tools upfront, agents can load specialized capabilities on demand.

### Why Skills?

1. **Context Efficiency**: Don't waste context on unused tool descriptions
2. **Specialization**: Skills can include detailed instructions for specific tasks
3. **Modularity**: Easy to add/remove capabilities
4. **Discoverability**: Agent can browse available skills

### SKILL.md Format

Skills are defined in markdown files with YAML frontmatter:

```markdown
---
name: skill-name
description: What this skill does
version: 1.0.0
tools:
  - tool1
  - tool2
---

# Skill Instructions

Detailed steps for how to use this skill...
```

In [44]:
# Let's look at the skills we created
skills_dir = Path("skills")

print("Available skills:")
for skill_dir in skills_dir.iterdir():
    if skill_dir.is_dir():
        skill_file = skill_dir / "SKILL.md"
        if skill_file.exists():
            content = skill_file.read_text()
            # Extract name and description from frontmatter
            lines = content.split("\n")
            name = ""
            desc = ""
            for line in lines:
                if line.startswith("name:"):
                    name = line.split(":", 1)[1].strip()
                if line.startswith("description:"):
                    desc = line.split(":", 1)[1].strip()
            print(f"  - {name}: {desc}")

Available skills:
  - meal-planning: Create personalized meal plans based on dietary needs and preferences
  - wellness-assessment: Assess user wellness goals and create personalized recommendations


In [45]:
# Read the wellness-assessment skill
skill_content = Path("skills/wellness-assessment/SKILL.md").read_text()
print(skill_content)

---
name: wellness-assessment
description: Assess user wellness goals and create personalized recommendations
version: 1.0.0
tools:
  - read_file
  - write_file
---

# Wellness Assessment Skill

You are conducting a comprehensive wellness assessment. Follow these steps:

## Step 1: Gather Information
Ask the user about:
- Current health goals (weight, fitness, stress, sleep)
- Any medical conditions or limitations
- Current exercise routine (or lack thereof)
- Dietary preferences and restrictions
- Sleep patterns and quality
- Stress levels and sources

## Step 2: Analyze Responses
Review the user's answers and identify:
- Primary wellness priority
- Secondary goals
- Potential barriers to success
- Existing healthy habits to build on

## Step 3: Create Assessment Report
Write a wellness assessment report to `workspace/wellness_assessment.md` containing:
- Summary of current wellness state
- Identified strengths
- Areas for improvement
- Recommended focus areas (prioritized)
- Suggeste

In [46]:
# Create a skill-aware tool
@tool
def load_skill(skill_name: str) -> str:
    """Load a skill's instructions for a specialized task.
    
    Available skills:
    - wellness-assessment: Assess user wellness and create recommendations
    - meal-planning: Create personalized meal plans
    
    Args:
        skill_name: Name of the skill to load
    
    Returns:
        Skill instructions
    """
    skill_path = Path(f"skills/{skill_name}/SKILL.md")
    if not skill_path.exists():
        available = [d.name for d in Path("skills").iterdir() if d.is_dir()]
        return f"Skill '{skill_name}' not found. Available: {', '.join(available)}"
    
    return skill_path.read_text()

print("Skill loader defined!")

Skill loader defined!


In [48]:
# Create an agent that can load and use skills
skill_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        load_skill,
        write_todos,
        update_todo,
        list_todos,
    ],
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a wellness assistant with access to specialized skills.

When a user asks for something that matches a skill:
1. Load the appropriate skill using load_skill()
2. Follow the skill's instructions carefully
3. Save outputs as specified in the skill

Available skills:
- wellness-assessment: For comprehensive wellness evaluations
- meal-planning: For creating personalized meal plans

If no skill matches, use your general wellness knowledge."""
)

print("Skill-aware agent created!")

Skill-aware agent created!


c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Pyt

In [49]:
# Test with a skill-appropriate request
TODO_STORE.clear()

result = skill_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "I'd like a wellness assessment. I'm a 35-year-old office worker who sits most of the day, has trouble sleeping, and wants to lose 15 pounds. I'm a religious Hindu and Non-vegetarian and have no major health conditions."
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## Your Wellness Assessment Complete!

Based on your profile as a 35-year-old office worker looking to lose 15 pounds with sleep challenges, I've created a comprehensive assessment. Here are your **immediate action items and recommendations**:

### 🎯 **Immediate Action Items (Start Today)**

1. **Set a Consistent Sleep Schedule**
   - Go to bed and wake up at the same time daily (even weekends)
   - Create a 30-minute wind-down routine before bed
   - Limit screen time 1 hour before sleep

2. **Implement Hourly Movement Breaks**
   - Set phone/computer reminders every 60 minutes
   - Stand, stretch, or walk for 2-3 minutes each hour
   - Take stairs instead of elevators when possible

3. **Start Food Awareness Tracking**
   - Log what and when you eat for one week
   - Note hunger levels and energy patterns
   - Identify your current eating habits without judgment

### 📅 **Short-term Goals (1-2 Weeks)**

1. **Establish Exercise Foundation**
   - Begin with 20-minute wal

## Task 9: Using deepagents-cli

The `deepagents-cli` provides an interactive terminal interface for working with Deep Agents.

### Installation

```bash
uv pip install deepagents-cli
# or
pip install deepagents-cli
```

### Key Features

| Feature | Description |
|---------|-------------|
| **Interactive Sessions** | Chat with your agent in the terminal |
| **Conversation Resume** | Pick up where you left off |
| **Human-in-the-Loop** | Approve or reject agent actions |
| **File System Access** | Agent can read/write to your filesystem |
| **Remote Sandboxing** | Run in isolated Docker containers |

### Basic Usage

```bash
# Start an interactive session
deepagents

# Resume a previous conversation
deepagents --resume

# Use a specific model
deepagents --model openai:gpt-4o

# Enable human-in-the-loop approval
deepagents --approval-mode full
```

### Example Session

```
$ deepagents

Welcome to Deep Agents CLI!

You: Create a 7-day meal plan for a vegetarian athlete

Agent: I'll create a comprehensive meal plan for you. Let me:
1. Research vegetarian athlete nutrition needs
2. Design balanced daily menus
3. Save the plan to a file

[Agent uses tools...]

Agent: I've created your meal plan! You can find it at:
workspace/vegetarian_athlete_meal_plan.md

You: /exit
```

In [50]:
# Check if CLI is installed
import subprocess

try:
    result = subprocess.run(["deepagents", "--version"], capture_output=True, text=True)
    print(f"deepagents-cli version: {result.stdout.strip()}")
except FileNotFoundError:
    print("deepagents-cli not installed. Install with:")
    print("  uv pip install deepagents-cli")
    print("  # or")
    print("  pip install deepagents-cli")

deepagents-cli not installed. Install with:
  uv pip install deepagents-cli
  # or
  pip install deepagents-cli


### Try It Yourself!

After installing the CLI, try these commands in your terminal:

```bash
# Basic interactive session
deepagents

# With a specific working directory
deepagents --workdir ./workspace

# See all options
deepagents --help
```

Sample prompts to try:
1. "Create a weekly workout plan and save it to a file"
2. "Research the health benefits of meditation and summarize in a report"
3. "Analyze my current diet and suggest improvements" (then provide details)

## Task 10: Building a Complete Deep Agent System

Now let's bring together all four elements to build a comprehensive "Wellness Coach" system:

1. **Planning**: Track multi-week wellness programs
2. **Context Management**: Store session notes and progress
3. **Subagent Spawning**: Delegate to specialists (exercise, nutrition, mindfulness)
4. **Long-term Memory**: Remember user preferences and history

In [51]:
# Define specialized wellness subagents
# Subagents inherit the backend from the parent, so they use the same workspace
exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Expert in exercise science, workout programming, and physical fitness. Use for exercise-related questions and plan creation.",
    "system_prompt": """You are an exercise specialist with expertise in:
- Workout programming for different fitness levels
- Exercise form and safety
- Progressive overload principles
- Recovery and injury prevention

Always consider the user's fitness level and any physical limitations.
Provide clear, actionable exercise instructions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Expert in nutrition science, meal planning, and dietary optimization. Use for food-related questions and meal plans.",
    "system_prompt": """You are a nutrition specialist with expertise in:
- Macro and micronutrient balance
- Meal planning and preparation
- Dietary restrictions and alternatives
- Nutrition timing for performance

Always respect dietary restrictions and preferences.
Focus on practical, achievable meal suggestions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Expert in stress management, sleep optimization, and mental wellness. Use for stress, sleep, and mental health questions.",
    "system_prompt": """You are a mindfulness and mental wellness specialist with expertise in:
- Stress reduction techniques
- Sleep hygiene and optimization
- Meditation and breathing exercises
- Work-life balance strategies

Be supportive and non-judgmental.
Provide practical techniques that can be implemented immediately.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

print("Specialist subagents defined!")

Specialist subagents defined!


In [61]:
# ------------------------------
# Deep Agents: Wellness Coach (Coordinator saves files safely; subagents NEVER write files)
# Paste this whole cell and run.
# ------------------------------

from pathlib import Path
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend

# ------------------------------
# 0) REQUIREMENTS CHECK
# ------------------------------
REQUIRED = [
    "write_todos", "update_todo", "list_todos", "TODO_STORE",
    "get_user_profile", "save_user_preference", "load_skill",
]
missing = [name for name in REQUIRED if name not in globals()]
if missing:
    raise NameError(
        f"Missing required objects from earlier notebook cells: {missing}\n"
        "Run the earlier Deep Agents notebook setup cells first (todo tools, memory tools, skills tools)."
    )

# ------------------------------
# 1) WORKSPACE BACKEND (SAFE)
# ------------------------------
WORKSPACE = Path.cwd() / "workspace"
WORKSPACE.mkdir(parents=True, exist_ok=True)

filesystem_backend = FilesystemBackend(
    root_dir=str(WORKSPACE),
    virtual_mode=True,
)

# ------------------------------
# 2) SAFE SAVE TOOL (ONLY filename allowed, always writes inside WORKSPACE)
# ------------------------------
@tool
def save_markdown(filename: str, content: str) -> str:
    """
    Save markdown content inside the WORKSPACE folder only.
    - filename must be a simple name like 'exercise_plan_user_alex.md' (no slashes, no '..')
    - content is the markdown text
    """
    if any(x in filename for x in ["/", "\\", ".."]):
        return "ERROR: filename must not contain '/', '\\\\', or '..'"

    if not filename.lower().endswith(".md"):
        filename = filename + ".md"

    out_path = WORKSPACE / filename
    out_path.write_text(content, encoding="utf-8")
    return f"Saved file: {out_path.name} ({out_path.stat().st_size} bytes)"

# ------------------------------
# 3) MODEL (Anthropic)
# ------------------------------
anthropic_model = init_chat_model("anthropic:claude-sonnet-4-20250514")

# ------------------------------
# 4) SUBAGENTS (NO FILE TOOLS)
# ------------------------------
exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Creates a safe 2-week workout plan based on schedule, goals, and constraints.",
    "model": anthropic_model,
    "tools": [], # IMPORTANT: no file tools to avoid permission errors
    "system_prompt": """
You are an Exercise Specialist.
Return ONLY markdown content (no tool calls) for a 2-week plan.

Requirements:
- 3 sessions/week, 30 minutes each
- Include warm-up, main set, cool-down
- Week 2 should slightly progress intensity/volume
- Keep it safe and practical
Output format:
# Exercise Plan (2 Weeks)
## Weekly Schedule
## Workouts (Session 1/2/3)
## Progression Notes
""".strip(),
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Creates a vegetarian 2-week nutrition plan with practical meals and grocery guidance.",
    "model": anthropic_model,
    "tools": [],
    "system_prompt": """
You are a Nutrition Specialist.
Return ONLY markdown content (no tool calls) for a 2-week vegetarian plan.

Requirements:
- Simple meal templates for breakfast/lunch/dinner + snacks
- Vegetarian protein strategy (beans/lentils/tofu/Greek yogurt if allowed, etc.)
- Grocery list suggestions
Output format:
# Nutrition Plan (2 Weeks, Vegetarian)
## Daily Meal Template
## 2-Week Meal Ideas (Week 1 + Week 2)
## Protein & Micronutrient Notes
## Grocery List
""".strip(),
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Creates a 2-week plan for stress management + sleep improvement with daily micro-practices.",
    "model": anthropic_model,
    "tools": [],
    "system_prompt": """
You are a Mindfulness & Sleep Specialist.
Return ONLY markdown content (no tool calls) for a 2-week stress + sleep plan.

Requirements:
- Daily 5–10 minute micro-practices
- Sleep hygiene checklist
- Stress regulation techniques (breathing, journaling, CBT-lite reframes, etc.)
Output format:
# Stress & Sleep Plan (2 Weeks)
## Daily Micro-Practices
## Sleep Hygiene Checklist
## Managing Work Stress
## Tracking & Adjustments
""".strip(),
}

# ------------------------------
# 5) COORDINATOR (Wellness Coach)
# ------------------------------
wellness_coach = create_deep_agent(
    model=anthropic_model,
    tools=[
        # Planning
        write_todos, update_todo, list_todos,
        # Long-term Memory
        get_user_profile, save_user_preference,
        # Skills
        load_skill,
        # Safe file saving
        save_markdown,
    ],
    backend=filesystem_backend,
    subagents=[exercise_specialist, nutrition_specialist, mindfulness_specialist],
    system_prompt="""
You are a Personal Wellness Coach coordinating a 2-week wellness program.

MANDATORY WORKFLOW:
1) Extract user_id from the user message.
2) Call get_user_profile(user_id=...).
3) Create todos for:
   - exercise plan
   - nutrition plan
   - mindfulness/sleep plan
   - save 3 files + create index
4) Delegate:
   - exercise-specialist -> returns markdown text for exercise plan
   - nutrition-specialist -> returns markdown text for vegetarian nutrition plan
   - mindfulness-specialist -> returns markdown text for stress+sleep plan
5) Save each specialist output using save_markdown with EXACT filenames:
   - exercise_plan_{user_id}.md
   - nutrition_plan_{user_id}.md
   - mindfulness_sleep_plan_{user_id}.md
6) Create an index markdown (your own text) summarizing the 2-week schedule and linking the 3 files:
   - program_{user_id}.md
   Save it using save_markdown.
7) Update todos to done as each step completes.
Finish by listing the 4 filenames created.

IMPORTANT:
- Respect vegetarian preference.
- Keep advice safe and actionable.
- Subagents do NOT write files; only you save files.
""".strip(),
)

print("✅ Wellness Coach created (safe file writes).")

# ------------------------------
# 6) TEST RUN
# ------------------------------
TODO_STORE.clear()

user_msg = """Hi! My user_id is user_alex. I'd like you to create a 2-week wellness program for me.

I want to focus on:
1. Building a consistent exercise routine (I can exercise 3x per week for 30 mins)
2. Improving my diet (remember I'm vegetarian)
3. Better managing my work stress and improving my sleep

Please create comprehensive plans for each area and save them as separate files I can reference."""
result = wellness_coach.invoke({"messages": [{"role": "user", "content": user_msg}]})

print("\n--- Wellness Coach response ---")
print(result["messages"][-1].content)

# ------------------------------
# 7) REVIEW RESULTS
# ------------------------------
print("\n" + "=" * 60)
print("FINAL TODO STATUS")
print("=" * 60)
print(list_todos.invoke({}))

print("\n" + "=" * 60)
print("GENERATED FILES (workspace)")
print("=" * 60)
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f" [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f" [DIR] {f.name}/")


c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Pyt

✅ Wellness Coach created (safe file writes).

--- Wellness Coach response ---
Excellent! Your comprehensive 2-week wellness program is complete! 🎉

## **📁 Files Created for Your Reference:**

1. **`program_user_alex.md`** - Your master program guide with integrated 2-week schedule
2. **`exercise_plan_user_alex.md`** - Complete workout routines (3x/week, 30 min sessions)
3. **`nutrition_plan_user_alex.md`** - Vegetarian meal plans, recipes, and shopping lists
4. **`mindfulness_sleep_plan_user_alex.md`** - Daily stress management and sleep optimization practices

## **🌟 What You Now Have:**

**Exercise Program**: 6 progressive workouts over 2 weeks, focusing on building consistent habits with bodyweight exercises that fit your 3x/week, 30-minute schedule.

**Vegetarian Nutrition Plan**: Complete meal planning with 14 days of breakfast, lunch, dinner, and snacks, plus detailed grocery lists, recipes, and nutritional guidance.

**Stress & Sleep Management**: Daily micro-practices (5-20 min

In [62]:
# Read one of the generated files
files = list(WORKSPACE.glob("*.md"))
if files:
    print(f"\nContents of {files[0].name}:")
    print("=" * 60)
    print(files[0].read_text()[:2000] + "..." if len(files[0].read_text()) > 2000 else files[0].read_text())


Contents of exercise_plan_user_alex.md:
# Exercise Plan for Alex (2 Weeks)

## Overview
This comprehensive 2-week exercise plan is designed to help you build a consistent fitness routine with variety and progressive challenge. Each workout is structured for 30 minutes and includes beginner-friendly options with clear progression paths.

## Weekly Schedule
- **Frequency:** 3 sessions per week
- **Duration:** 30 minutes per session
- **Structure:** Monday, Wednesday, Friday (adjust days as needed)
- **Rest Days:** Active recovery or complete rest

---

## Week 1

### Session 1: Full Body Strength (Monday)

**Warm-Up (5 minutes)**
- Arm circles: 30 seconds each direction
- Leg swings: 30 seconds each leg
- Bodyweight squats: 1 minute
- Marching in place: 1 minute
- Dynamic stretching: 1.5 minutes

**Main Set (20 minutes)**
1. **Push-ups** (or knee push-ups)
   - 3 sets × 8-12 reps
   - Rest: 30 seconds between sets
   - Form cue: Keep body straight, lower chest to ground

2. **Bodyweight

---
## ❓ Question #3:

What are the key considerations when designing **subagent configurations**?

Consider:
- When should subagents share tools vs have distinct tools?
- How do you decide which model to use for each subagent?
- What's the right granularity for subagent specialization?

##### Answer:
**Tools: shared vs distinct

1. Share tools when they’re safe + generic and needed by many agents (e.g., read-only KB retrieval, basic file read, simple calculator).

2. Distinct tools when you want safety + control:

    a. limit a tool to the agent that truly needs it (SQL tool only to “data agent”)

    b. prevent misuse (web search only to “research agent”)

    c. reduce prompt/tool confusion (fewer tools per agent = better tool selection)


### Rule of thumb: give each subagent the minimum tools needed (principle of least privilege).


Choosing the model per subagent

Use stronger models for:

1. routing/orchestration (higher stakes decisions)

2. complex synthesis (combining outputs, handling ambiguity)

3. safety-sensitive reasoning


Use cheaper/faster models for:

1. templated writing, formatting markdown

2. extracting structured facts from a known document

3. simple plan generation with constraints already provided


Also consider context length: subagents that read long docs may need models with long-context support.


Right granularity of specialization

1. Too broad (“wellness agent”) → becomes a generalist and loses quality.

2. Too narrow (“agent for 10-minute breathing only”) → orchestration overhead explodes.

Sweet spot: domain-level specialists aligned to your product areas:

exercise, nutrition, sleep/stress, habits


**

## ❓ Question #4:

For a **production wellness application** using Deep Agents, what would you need to add?

Consider:
- Safety guardrails for health advice
- Persistent storage (not in-memory)
- Multi-user support and isolation
- Monitoring and observability
- Cost management with subagents

##### Answer:
1. Safety guardrails:

    a. A medical safety policy in system prompts: no diagnosis, no medication changes, escalate to clinician for red flags.

    b. “Contraindication checks” for allergies/conditions/meds before recommendations.

    c. Restricted tools: web search only when needed + source filtering; avoid untrusted sources.

    d. Safety testing: red-team prompts + automatic checks.


2. Persistent storage:

    a. Replace in-memory TODO/profile stores with real storage:

    b. Postgres (users, preferences, plans, todo state)

    c. Object storage (S3) for generated markdown plans

    d. Vector DB (optional) for retrieval over user history/docs


3. Add versioning for plans (“program_v3”) and audit logs.


4. Multi-user support + isolation:

    a. Every request must carry user_id / tenant_id.

    b. Strict namespace separation for:

    c. files (per-user workspace paths)

    d. memory keys (preferences, conditions)

    d. vector indexes / caches


5. Access control rules so one user can never see another’s data.


6. Monitoring & observability:

    a. Traces per run: which subagents fired, tool calls, latency per step.

    b. Metrics: success rate, timeouts, tool errors, token usage, cache hit rate.

    c. Logging of outputs + safety events (with PII handling).

7. Human feedback loop (“thumbs up/down” + reasons) stored for improvement.


8. Cost management with subagents:

    a. Limit fan-out: don’t call all subagents by default—route to only what’s needed.

    b. Use model tiers: big model for coordinator, smaller models for specialists where possible.

    c. Add budgets: max subagent calls per request, max tokens, max retries.

    d. Cache/reuse outputs (e.g., weekly plan doesn’t need regeneration every turn).

    e. Summarize context aggressively to reduce tokens and avoid “context rot.”

---
## 🏗️ Activity #2: Build a Wellness Coach Agent

Build your own wellness coach that uses all 4 Deep Agent elements.

### Requirements:
1. **Planning**: Create todos for a 30-day wellness challenge
2. **Context Management**: Store daily check-in notes
3. **Subagents**: At least 2 specialized subagents
4. **Memory**: Remember user preferences across interactions

### Challenge:
Create a "30-Day Wellness Challenge" system that:
- Generates a personalized 30-day plan
- Tracks daily progress
- Adapts recommendations based on feedback
- Saves a weekly summary report

In [64]:
# ============================================================
# Activity #2 — 30-Day Wellness Challenge Coach (Deep Agents)
# ============================================================

from __future__ import annotations
from pathlib import Path
from datetime import datetime
from typing import Optional
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend

# ------------------------------
# 0) REQUIREMENTS CHECK
# ------------------------------
REQUIRED = [
    "write_todos", "update_todo", "list_todos", "TODO_STORE",
    "get_user_profile", "save_user_preference", "load_skill",
]
missing = [name for name in REQUIRED if name not in globals()]
if missing:
    raise NameError(
        f"Missing required objects from earlier notebook cells: {missing}\n"
        "Run the earlier Deep Agents notebook setup cells first (todo tools, memory tools, skills tools)."
    )

# ------------------------------
# 1) WORKSPACE BACKEND
# ------------------------------
WORKSPACE = Path.cwd() / "workspace"
WORKSPACE.mkdir(parents=True, exist_ok=True)

backend = FilesystemBackend(root_dir=str(WORKSPACE), virtual_mode=True)

# ------------------------------
# 2) SAFE FILE TOOLS (Coordinator-only)
# ------------------------------
def _safe_user_id(user_id: str) -> str:
    return "".join(c for c in user_id.strip() if c.isalnum() or c in ["_", "-"]).strip("_-") or "user"

@tool
def save_text(path: str, content: str) -> str:
    """
    Save content to a workspace-relative path ONLY.
    path must not contain absolute paths or '..'.
    Example: 'users/user_alex/checkins/2026-02-08.md'
    """
    if ":" in path or path.startswith(("/", "\\")) or ".." in path or path.strip() == "":
        return "ERROR: Invalid path. Use a relative workspace path with no '..' or absolute paths."

    full = WORKSPACE / path
    full.parent.mkdir(parents=True, exist_ok=True)
    full.write_text(content, encoding="utf-8")
    return f"Saved: {path} ({full.stat().st_size} bytes)"

@tool
def append_text(path: str, content: str) -> str:
    """
    Append content to a workspace-relative file ONLY.
    """
    if ":" in path or path.startswith(("/", "\\")) or ".." in path or path.strip() == "":
        return "ERROR: Invalid path. Use a relative workspace path with no '..' or absolute paths."

    full = WORKSPACE / path
    full.parent.mkdir(parents=True, exist_ok=True)
    with full.open("a", encoding="utf-8") as f:
        f.write(content)
    return f"Appended: {path}"

@tool
def list_user_files(user_id: str) -> str:
    """List files under workspace/users/<user_id>/"""
    uid = _safe_user_id(user_id)
    root = WORKSPACE / "users" / uid
    if not root.exists():
        return f"No files yet for user_id={uid}"
    files = []
    for p in sorted(root.rglob("*")):
        if p.is_file():
            rel = p.relative_to(WORKSPACE).as_posix()
            files.append(f"- {rel} ({p.stat().st_size} bytes)")
    return "\n".join(files) if files else f"No files yet for user_id={uid}"

# ------------------------------
# 3) SUBAGENTS (2 specialists; NO file tools)
# ------------------------------
model = init_chat_model("anthropic:claude-sonnet-4-20250514")

exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Designs safe, practical exercise plans and daily movement challenges.",
    "model": model,
    "tools": [],
    "system_prompt": """
You are an Exercise Specialist for a 30-day wellness challenge.
Return ONLY markdown content (no tool calls).
Create:
- A weekly structure (Weeks 1–4) and a daily template
- Options for beginner/moderate intensity
- Safety notes and progression rules
Keep sessions short and realistic.
""".strip(),
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Designs nutrition habits and vegetarian-friendly meal templates if needed.",
    "model": model,
    "tools": [],
    "system_prompt": """
You are a Nutrition Specialist for a 30-day wellness challenge.
Return ONLY markdown content (no tool calls).
Create:
- Daily nutrition habit targets
- Meal templates (include vegetarian option if user prefers)
- Simple grocery + prep guidance
Keep it practical and habit-based.
""".strip(),
}

# Optional third specialist (helps with stress/sleep)
mindset_specialist = {
    "name": "mindset-specialist",
    "description": "Designs stress management, sleep hygiene, and daily mental wellness practices.",
    "model": model,
    "tools": [],
    "system_prompt": """
You are a Mindset & Sleep Specialist for a 30-day wellness challenge.
Return ONLY markdown content (no tool calls).
Create:
- Daily 5–10 min practices
- Sleep hygiene checklist
- Work-stress coping plan
Keep it supportive and actionable.
""".strip(),
}

# ------------------------------
# 4) COORDINATOR AGENT (Wellness Coach)
# ------------------------------
wellness_challenge_coach = create_deep_agent(
    model=model,
    backend=backend,
    subagents=[exercise_specialist, nutrition_specialist, mindset_specialist],
    tools=[
        # Planning
        write_todos, update_todo, list_todos,
        # Memory
        get_user_profile, save_user_preference,
        # Skills (if your notebook uses skill files)
        load_skill,
        # Context management via files
        save_text, append_text, list_user_files,
    ],
    system_prompt="""
You are a 30-Day Wellness Challenge Coach. You MUST use all 4 Deep Agent elements:
1) Planning: create and update todos for the 30-day challenge.
2) Context management: store daily check-ins and weekly summaries in files.
3) Subagents: delegate to specialists for exercise/nutrition/mindset.
4) Memory: read user profile and store user preferences for future interactions.

You support TWO main user intents:
A) "Create my 30-day plan" (initial setup)
B) "Daily check-in" (ongoing tracking + adaptation)

FILE CONVENTION (required):
All user files live under: users/<user_id>/
- users/<user_id>/plan_30_day.md
- users/<user_id>/checkins/YYYY-MM-DD.md
- users/<user_id>/weekly_summaries/week_<N>_summary.md

WORKFLOW — Initial Setup ("create plan"):
1) Extract user_id. Call get_user_profile(user_id=...).
2) Ask for missing preferences only if truly required; otherwise infer sensible defaults.
3) Create todos for:
   - exercise plan section
   - nutrition plan section
   - mindset/sleep section
   - integrate into final 30-day plan
   - save plan file
4) Delegate to exercise-specialist, nutrition-specialist, mindset-specialist.
5) Integrate outputs into one cohesive markdown plan:
   - Overview, safety notes
   - Week 1–4 structure + daily template
   - How to track progress
   - How to adapt when user struggles
6) Save plan: users/<user_id>/plan_30_day.md using save_text.
7) Update todos to done and tell the user the saved filename + how to do daily check-ins.

WORKFLOW — Daily Check-in ("check-in"):
1) Extract user_id and date (use today if not provided).
2) Store the check-in note in: users/<user_id>/checkins/YYYY-MM-DD.md
3) Decide week number (1–4) based on day-of-challenge if provided; otherwise infer from existing checkins count.
4) Provide 1–3 adaptive recommendations for tomorrow (exercise/nutrition/mindset), based on the check-in.
5) Every 7 check-ins (or on user request), write a weekly summary report to:
   users/<user_id>/weekly_summaries/week_<N>_summary.md
   Include: wins, barriers, trend notes, next-week tweaks.

MEMORY RULE:
- Save stable preferences (vegetarian, schedule constraints, disliked foods, injuries) using save_user_preference.
- Do NOT store overly sensitive medical info unless user explicitly provides it.

IMPORTANT:
- Keep advice safe and non-medical.
- Subagents do not write files; only you do via save_text/append_text.
""".strip(),
)

print("✅ 30-Day Wellness Challenge Coach created!")

# ------------------------------
# 5) QUICK TESTS
# ------------------------------
TODO_STORE.clear()

# Test 1: create plan
setup_prompt = """Hi! My user_id is Krishanu.
Please create a personalized 30-Day Wellness Challenge for me.
Preferences:
- Indian Vegetarian
- I can do 30 minutes exercise 3x/week
- Main goals: energy, lower stress, better sleep
Save the plan so I can reference it later."""
out = wellness_challenge_coach.invoke({"messages": [{"role": "user", "content": setup_prompt}]})
print("\n--- Setup Response ---\n", out["messages"][-1].content)

print("\n--- TODOS ---")
print(list_todos.invoke({}))

print("\n--- User Files ---")
print(list_user_files.invoke({"user_id": "Krishanu"}))

# Test 2: daily check-in
checkin_prompt = """Hi! My user_id is Krishanu.
Daily check-in (today):
- Sleep: 5/10 (woke up twice)
- Stress: high (deadlines)
- Exercise: none
- Nutrition: decent, but snacked late
What should I do tomorrow? Also save this check-in."""
out2 = wellness_challenge_coach.invoke({"messages": [{"role": "user", "content": checkin_prompt}]})
print("\n--- Check-in Response ---\n", out2["messages"][-1].content)

print("\n--- User Files After Check-in ---")
print(list_user_files.invoke({"user_id": "Krishanu"}))



c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\SinhaK\AppData\Local\miniconda3\envs\agentenv312\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: typing.NotRequired is not a Pyt

✅ 30-Day Wellness Challenge Coach created!

--- Setup Response ---
 🎉 **Your Personalized 30-Day Wellness Challenge is Complete, Krishanu!**

I've successfully created and saved your comprehensive wellness plan to: **`users/Krishanu/plan_30_day.md`**

## What You Now Have:

✅ **Culturally-Aligned Plan**: Designed specifically for your Indian vegetarian lifestyle
✅ **Time-Efficient Structure**: 30-minute workouts, 3x/week + daily 5-15 minute mindfulness practices  
✅ **Goal-Focused Approach**: Every element targets your specific goals of higher energy, lower stress, and better sleep
✅ **Progressive Design**: 4 weeks of increasing challenge with built-in adaptation strategies
✅ **Holistic Integration**: Exercise, nutrition, and mindfulness work together synergistically

## Your Plan Includes:
- **🏋️ Exercise Program**: Progressive 30-minute sessions with modifications for all levels
- **🥗 Nutrition Guide**: Traditional Indian vegetarian meals optimized for your wellness goals
- **🧘 Mindf

---
## Summary

In this session, we explored **Deep Agents** and their four key elements:

| Element | Purpose | Implementation |
|---------|---------|----------------|
| **Planning** | Track complex tasks | `write_todos`, `update_todo`, `list_todos` |
| **Context Management** | Handle large contexts | File system tools, automatic offloading |
| **Subagent Spawning** | Delegate to specialists | `task` tool with custom configs |
| **Long-term Memory** | Remember across sessions | LangGraph Store integration |

### Key Takeaways:

1. **Deep Agents handle complexity** - Unlike simple tool loops, they can manage long-horizon, multi-step tasks
2. **Planning is context engineering** - Todo lists and files aren't just organization—they're extended memory
3. **Subagents prevent context bloat** - Delegation keeps the main agent focused and efficient
4. **Skills enable progressive disclosure** - Load capabilities on-demand instead of upfront
5. **The CLI makes interaction natural** - Interactive sessions with conversation resume

### Deep Agents vs Traditional Agents

| Aspect | Traditional Agent | Deep Agent |
|--------|-------------------|------------|
| Task complexity | Simple, single-step | Complex, multi-step |
| Context management | All in conversation | Files + summaries |
| Delegation | None | Subagent spawning |
| Memory | Within thread | Across sessions |
| Planning | Implicit | Explicit (todos) |

### When to Use Deep Agents

**Use Deep Agents when:**
- Tasks require multiple steps or phases
- Context would overflow in a simple loop
- Specialization would improve quality
- Users need to resume sessions
- Long-term memory is valuable

**Use Simple Agents when:**
- Tasks are straightforward Q&A
- Single tool call suffices
- Context fits easily
- No need for persistence

### Further Reading

- [Deep Agents Documentation](https://docs.langchain.com/oss/python/deepagents/overview)
- [Deep Agents GitHub](https://github.com/langchain-ai/deepagents)
- [Context Management Blog Post](https://www.blog.langchain.com/context-management-for-deepagents/)
- [Building Multi-Agent Applications](https://www.blog.langchain.com/building-multi-agent-applications-with-deep-agents/)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)